<a href="https://colab.research.google.com/github/svmmerfantasy/Bangkit-2023/blob/main/Collaborative_Filtering_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from tensorflow import keras

#Preparing Dataset


#CLEANING DATASET

In [7]:
mentee = pd.read_csv("Mentees.csv")
mentor = pd.read_csv("Mentors.csv")
rating = pd.read_csv("Ratings.csv")

In [8]:
cols_mente = {"ID MENTEE" : "MenteeID", "SUBJECT" : "Subject" ,"NAME" : "MenteeName", "LOCATION" : "Location"}
mentee.rename(columns=cols_mente, inplace=True)
mentee_df = mentee.loc[0:1499, 'MenteeID' : 'LONGITUDE']

In [5]:
cols_mentor = {"ID MENTOR" :"TutorID", "NAME" : "TutorName", "LOCATION" : "Location", "MATKUL" : "Subjects"}
mentor.rename(columns=cols_mentor, inplace=True)
temporary = mentor.drop(columns=['SUBJECT'])
mentor_df = temporary.loc[0:499, 'TutorID' : 'LONGITUDE']

In [9]:
cols_rating = {"ID MENTEE" : "MenteeID", "ID MENTOR" : "TutorID", "RATING" : "Rating"}
rating.rename(columns=cols_rating, inplace=True)
rating_df = rating.loc[0:12625, 'MenteeID' : 'Rating']
rating_df['Rating'] = rating_df['Rating'].astype(np.int64)

In [10]:
popular_tutor_rating_df = rating_df.merge(mentor_df, on="TutorID")
popular_tutor_rating_df

,MenteeID,TutorID,Rating,TutorName,Subjects,Location,SUBDISTRICT,LATITUDE,LONGITUDE
0,B36652,A16227,3,Andrianto,Ekonomi,Jakarta,JOHAR BARU,-6.183050,106.854320
1,B63077,A16227,5,Andrianto,Ekonomi,Jakarta,JOHAR BARU,-6.183050,106.854320
2,B14364,A16227,5,Andrianto,Ekonomi,Jakarta,JOHAR BARU,-6.183050,106.854320
3,B61521,A16227,4,Andrianto,Ekonomi,Jakarta,JOHAR BARU,-6.183050,106.854320
4,B62409,A16227,5,Andrianto,Ekonomi,Jakarta,JOHAR BARU,-6.183050,106.854320
...,...,...,...,...,...,...,...,...,...
12621,B29822,A17248,3,Junaeri,Matematika,Yogyakarta,PAKUALAMAN,-7.800546,110.375171
12622,B64162,A17248,3,Junaeri,Matematika,Yogyakarta,PAKUALAMAN,-7.800546,110.375171
12623,B32844,A17248,4,Junaeri,Matematika,Yogyakarta,PAKUALAMAN,-7.800546,110.375171
12624,B97893,A17248,5,Junaeri,Matematika,Yogyakarta,PAKUALAMAN,-7.800546,110.375171


#PREPROCESSING DATASET

##Create new dataframe to feed model

In [11]:
popular_tutor_rating = popular_tutor_rating_df[['MenteeID', 'TutorID', 'Subjects', 'Rating']]
popular_tutor_rating

,MenteeID,TutorID,Subjects,Rating
0,B36652,A16227,Ekonomi,3
1,B63077,A16227,Ekonomi,5
2,B14364,A16227,Ekonomi,5
3,B61521,A16227,Ekonomi,4
4,B62409,A16227,Ekonomi,5
...,...,...,...,...
12621,B29822,A17248,Matematika,3
12622,B64162,A17248,Matematika,3
12623,B32844,A17248,Matematika,4
12624,B97893,A17248,Matematika,5


#COLLABORATIVE FILTERING USING TENSORFLOW

###SPLIT DATASET

In [12]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(popular_tutor_rating, test_size=0.2, random_state=42)

###MAP MENTEEID AND TUTORID

In [13]:
# Get the unique mentee and tutor IDs
mentee_ids = popular_tutor_rating['MenteeID'].unique()

# Create mappings from IDs to indices and vice versa
mentee_to_index = {id: index for index, id in enumerate(mentee_ids)}
index_to_mentee = {index: id for index, id in enumerate(mentee_ids)}

# Get the unique TutorID values
tutor_ids = popular_tutor_rating['TutorID'].unique()

# Create a mapping from TutorID to index
tutor_to_index = {id: index for index, id in enumerate(tutor_ids)}
index_to_tutor = {index: id for index, id in enumerate(tutor_ids)}

###MASKING TRAIN DATA & TEST DATA

In [14]:
# Masking for training data
df_train = popular_tutor_rating.copy()

for i in range(test_data.shape[0]):
  df_train.loc[(df_train.MenteeID == test_data.iloc[[i]].MenteeID.values[0]) & (df_train.TutorID == test_data.iloc[[i]].TutorID.values[0]), 'Rating'] = 0

In [15]:
# Making rating 0

df_test = popular_tutor_rating.copy()

for col in df_test.Rating :
  df_test['Rating'].values[:] = 0

In [16]:
# Masking for testing data

for i in range(test_data.shape[0]):
  df_test.loc[(df_test.MenteeID == test_data.iloc[[i]].MenteeID.values[0]) & (df_test.TutorID == test_data.iloc[[i]].TutorID.values[0]), 'Rating'] = test_data.iloc[[i]].Rating.values[0]

In [17]:
user_tutor_matrix_train = pd.pivot_table(
    data=df_train,
    values='Rating',
    index='MenteeID',
    columns='TutorID',
    # aggfunc=np.mean,  # Use the mean rating as the aggregation function
    fill_value=0  # Fill missing values with 0
)
user_tutor_matrix_train

TutorID,A11289,A11389,A11607,A11627,A11676,A11963,A12059,A12073,A12393,A12453,...,A97882,A97962,A98249,A98365,A98765,A98819,A98965,A99021,A99291,A99575
MenteeID,,,,,,,,,,,,,,,,,,,,,
B11181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B11233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B11260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B11380,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
B11407,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B99514,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B99594,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B99678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
user_tutor_matrix_test = pd.pivot_table(
    data=df_test,
    values='Rating',
    index='MenteeID',
    columns='TutorID',
    # aggfunc=np.mean,  # Use the mean rating as the aggregation function
    fill_value=0  # Fill missing values with 0
)
user_tutor_matrix_test

TutorID,A11289,A11389,A11607,A11627,A11676,A11963,A12059,A12073,A12393,A12453,...,A97882,A97962,A98249,A98365,A98765,A98819,A98965,A99021,A99291,A99575
MenteeID,,,,,,,,,,,,,,,,,,,,,
B11181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B11233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B11260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B11380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B11407,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B99514,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B99594,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B99678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
data_test = pd.pivot_table(
    data=popular_tutor_rating,
    values='Rating',
    index='MenteeID',
    columns='TutorID',
    # aggfunc=np.mean,  # Use the mean rating as the aggregation function
    fill_value=0  # Fill missing values with 0
)
# Generate random ratings for empty values
data_test_filled = data_test.applymap(lambda x: np.random.randint(1, 6) if x == 0 else x)
data_test_filled

TutorID,A11289,A11389,A11607,A11627,A11676,A11963,A12059,A12073,A12393,A12453,...,A97882,A97962,A98249,A98365,A98765,A98819,A98965,A99021,A99291,A99575
MenteeID,,,,,,,,,,,,,,,,,,,,,
B11181,5,4,4,5,1,1,5,1,1,3,...,5,2,1,2,1,2,1,2,4,4
B11233,4,2,1,2,4,4,4,2,1,5,...,2,5,4,2,5,1,4,2,2,3
B11260,5,4,3,5,2,4,1,4,5,1,...,3,5,2,4,1,3,4,4,1,2
B11380,1,3,2,2,3,4,1,1,1,3,...,1,4,4,1,2,3,1,3,2,2
B11407,4,2,3,2,5,4,4,5,5,2,...,3,4,1,5,1,4,1,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B99514,4,2,5,5,3,5,2,2,1,3,...,3,5,3,4,4,2,3,5,1,1
B99594,5,4,1,4,3,1,5,1,5,1,...,3,5,1,4,3,4,3,1,3,5
B99678,5,5,5,2,5,1,5,5,3,2,...,4,2,2,2,3,4,2,2,2,1


##BUILD A MODEL



In [52]:
class CollaborativeFilteringModel(tf.Module):
    def __init__(self, P, Q):
        super(CollaborativeFilteringModel, self).__init__()
        self.P = tf.Variable(P, name='P')
        self.Q = tf.Variable(Q, name='Q')

    @tf.function(input_signature=[tf.TensorSpec(shape=(None,None), dtype=tf.float32)])
    def predict(self, X):
        # Perform predictions using the learned latent features
        pred_R = tf.matmul(self.P, tf.transpose(self.Q))

        # Calculate the predicted ratings matrix
        predictions = tf.identity(pred_R)

        return {'predictions': predictions}

    def fit(self, X_train, num_epochs, learning_rate, regularization):
        # Normalize the input ratings matrix
        X_train = np.array(X_train, dtype=np.float32)
        mean_rating = np.mean(X_train)
        X_train -= mean_rating

        # Get the number of users and items
        num_users, num_items = X_train.shape

        # Define the input placeholder
        R = tf.constant(X_train, dtype=tf.float32)

        # Define the optimizer
        optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

        @tf.function
        def compute_loss(R, P, Q):
            pred_R = tf.matmul(P, tf.transpose(Q))
            cur_loss = tf.reduce_sum(tf.square(R - pred_R))
            regularization_term = tf.reduce_sum(tf.square(P)) + tf.reduce_sum(tf.square(Q))
            cur_loss += regularization * regularization_term
            return cur_loss

        for epoch in range(num_epochs):
            with tf.GradientTape() as tape:
                cur_loss = compute_loss(R, self.P, self.Q)

            gradients = tape.gradient(cur_loss, [self.P, self.Q])
            optimizer.apply_gradients(zip(gradients, [self.P, self.Q]))

            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {cur_loss.numpy()}")

In [53]:
## Optional
    # def update_and_train(self, mentee_idx, mentor_idx, ratings, num_epochs, learning_rate, regularization):
    #         # Update the training data with the new ratings
    #     X_train_new = np.copy(self.X_train)
    #     for mentor_idx, rating in zip(mentor_idx, ratings):
    #         X_train_new[mentee_idx, mentor_idx] = rating

    #         # Perform training with the updated data
    #     self.fit(X_train_new, num_epochs, learning_rate, regularization)

In [54]:
def build_model(X_train, num_epochs, learning_rate, regularization, latent_dim):

    # Get the number of users and items
    num_users, num_items = X_train.shape

    # Initialize the user and item latent feature matrices
    P = tf.Variable(tf.random.normal([num_users, latent_dim]))
    Q = tf.Variable(tf.random.normal([num_items, latent_dim]))

    # Create an instance of CollaborativeFilteringModel
    model = CollaborativeFilteringModel(P, Q)

    # Fit the model
    model.fit(X_train, num_epochs, learning_rate, regularization)

    return model

In [68]:
# Set the hyperparameters
num_epochs = 200
learning_rate = 0.02
regularization = 0.04
latent_dim = 150
# early_stopping=True
#early_stopping=True, patience=5

##Test Model

In [69]:
model = build_model(user_tutor_matrix_train, num_epochs, learning_rate, regularization, latent_dim)

Epoch 10/200, Loss: 59172820.0
Epoch 20/200, Loss: 29445492.0
Epoch 30/200, Loss: 15145479.0
Epoch 40/200, Loss: 8060078.0
Epoch 50/200, Loss: 4413129.5
Epoch 60/200, Loss: 2482552.75
Epoch 70/200, Loss: 1443571.25
Epoch 80/200, Loss: 878243.4375
Epoch 90/200, Loss: 567358.0625
Epoch 100/200, Loss: 393959.53125
Epoch 110/200, Loss: 295289.0
Epoch 120/200, Loss: 237716.875
Epoch 130/200, Loss: 203167.765625
Epoch 140/200, Loss: 181809.0
Epoch 150/200, Loss: 168199.6875
Epoch 160/200, Loss: 159265.546875
Epoch 170/200, Loss: 153227.34375
Epoch 180/200, Loss: 149028.671875
Epoch 190/200, Loss: 146025.234375
Epoch 200/200, Loss: 143813.6875


In [70]:
def calculate_mse_rmse(model, X_test):
    pred_R = model.predict(X_test)['predictions']
    actual_R = np.array(X_test, dtype=np.float32)
    mse = tf.reduce_mean(tf.square(actual_R - pred_R))
    rmse = tf.sqrt(mse)
    return mse.numpy(), rmse.numpy()

In [71]:
mse, rmse = calculate_mse_rmse(model, user_tutor_matrix_test)
mse, rmse

(0.10510359, 0.32419685)

In [27]:
user_tutor_matrix_train

TutorID,A11289,A11389,A11607,A11627,A11676,A11963,A12059,A12073,A12393,A12453,...,A97882,A97962,A98249,A98365,A98765,A98819,A98965,A99021,A99291,A99575
MenteeID,,,,,,,,,,,,,,,,,,,,,
B11181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B11233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B11260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B11380,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
B11407,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B99514,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B99594,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B99678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
## =======OPTIONAL=======

# # ID pengguna yang memberikan banyak rating
# user_id = 'B11260'

# # Daftar ID item dan rating yang diberikan oleh pengguna
# item_ids = ['A11389', 'A11627', 'A12059']  # Daftar ID item
# ratings = [5, 1, 4]  # Daftar nilai rating

# mentee_idx = mentee_to_index[user_id]
# mentor_idx = [tutor_to_index[item_id] for item_id in item_ids]
# model.update_and_train(mentee_idx, mentor_idx, ratings, num_epochs, learning_rate, regularization)

AttributeError: ignored

#Save Model

In [29]:
import os
print(os.getcwd())

/content


In [30]:
# Menyimpan model dengan signature
tf.saved_model.save(
    model,
    'collaborative_filtering_model',
    signatures={
        'predict': model.predict.get_concrete_function(
            tf.TensorSpec(shape=(None, None), dtype=tf.float32))
    }
)

##Testing to Make Recommendation (Cloud Computing Code)

In [72]:
loaded_model = tf.saved_model.load('collaborative_filtering_model')

In [73]:
# Generate predictions using the loaded model
test_pred = tf.matmul(loaded_model.P, tf.transpose(loaded_model.Q))

# Convert user_tutor_matrix_test to numpy array
X_test = np.array(user_tutor_matrix_test, dtype=np.float32)
# Calculate the MSE and RMSE for testing
mse = tf.reduce_mean(tf.square(X_test - test_pred))
rmse = tf.sqrt(mse)
print("MSE on loaded model:", mse.numpy())
print("RMSE on loaded model:", rmse.numpy())

MSE on loaded model: 0.13520788
RMSE on loaded model: 0.36770624


In [74]:
new_predictions = loaded_model.predict(user_tutor_matrix_test)
predictions_array = new_predictions['predictions'].numpy()

In [75]:
def model_recommend(mentee_id, pred, mentee_to_index, index_to_tutor, top_k=10):
    # Get the index of the specified mentee
    mentee_index = mentee_to_index.get(mentee_id)

    if mentee_index is None:
        print("Mentee ID not found.")
        return []

    # Get the predicted ratings for the specified mentee from the predictions array
    mentee_pred_ratings = pred[mentee_index]

    # Sort the predicted ratings in descending order
    top_indices = np.argsort(mentee_pred_ratings)[::-1][:top_k]

    # Map the indices back to tutor IDs
    top_tutors = [index_to_tutor[i] for i in top_indices]

    return top_tutors

In [87]:
mentee_id = 'B36652'
# Get the top recommended tutors for the specified mentee
recommended_tutors = model_recommend(mentee_id, predictions_array, mentee_to_index, index_to_tutor, top_k=10)

print("Top recommended tutors for mentee", mentee_id, ":")
for tutor_id in recommended_tutors:
    print(tutor_id)

Top recommended tutors for mentee B36652 :
A48510
A29518
A60327
A73404
A47501
A14632
A50565
A11389
A21475
A34900


##To Make Sure the model give good enough recommendation

In [77]:
popular_tutor_rating.loc[popular_tutor_rating['MenteeID']=='B36652']

,MenteeID,TutorID,Subjects,Rating
0,B36652,A16227,Ekonomi,3
52,B36652,A97882,Biologi,5
98,B36652,A39784,Geografi,5
136,B36652,A98819,Kimia,3
138,B36652,A73041,Biologi,3
347,B36652,A62677,Kimia,5
386,B36652,A30458,Matematika,3
428,B36652,A90160,Bahasa Inggris,4
524,B36652,A56161,Sosiologi,3
562,B36652,A73598,Sosiologi,3


In [88]:
popular_tutor_rating.loc[popular_tutor_rating['TutorID']=='A14632']

,MenteeID,TutorID,Subjects,Rating
8239,B34224,A14632,Bahasa Indonesia,4
8240,B68534,A14632,Bahasa Indonesia,4
8241,B95146,A14632,Bahasa Indonesia,4
8242,B75805,A14632,Bahasa Indonesia,3
8243,B18541,A14632,Bahasa Indonesia,3
8244,B11582,A14632,Bahasa Indonesia,5
8245,B29337,A14632,Bahasa Indonesia,3
8246,B46912,A14632,Bahasa Indonesia,5
8247,B68262,A14632,Bahasa Indonesia,4
8248,B75530,A14632,Bahasa Indonesia,4


#TF LITE DEPLOY

In [ ]:
# Load the saved model
model = tf.saved_model.load('collaborative_filtering_model')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model('collaborative_filtering_model')
tflite_model = converter.convert()

# Save the TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)